In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
movies = pd.read_csv(r"C:\Users\JAYANKONDAN\Documents\college hw\Netflix_Dataset_Movie.csv")
ratings = pd.read_csv(r"C:\Users\JAYANKONDAN\Documents\college hw\Netflix_Dataset_Rating.csv")

In [2]:
movies.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


In [3]:
ratings.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


In [117]:
ratings['User'] = ratings['User_ID'].astype('category').cat.codes
ratings['Movie'] = ratings['Movie_ID'].astype('category').cat.codes
ratings['Rating'] = ratings['Rating'].transform( lambda x: x-3 if x!=0 else 0)
ratings.head()

,User_ID,Rating,Movie_ID,User,Movie
0,712664,2,3,38885,0
1,1331154,1,3,72494,0
2,2632461,0,3,142536,0
3,44937,2,3,2467,0
4,656399,1,3,35784,0


In [118]:
mat = sparse.coo_matrix((ratings['Rating'], (ratings['User'],ratings['Movie'])))

In [119]:
mat1 = mat.tocsr()

# JACCARD similarity

In [120]:
def jacsim(user):
    col = mat1[user].toarray().flatten()
    
    a = (col!=0).astype(int)
    b = (mat1!=0).astype(int)
    sim_users = a@b.T
    
    s1 = np.array(np.sum(b,axis=1)).flatten()
    s2 = np.sum(a)

    sim_users = sim_users/(s1 +s2 -sim_users)
    ind = (-sim_users).argsort()
    
    return ind

In [121]:
jacsim(75)[:10]

array([   75, 31668, 66257, 55320, 60936, 87492, 77865, 76088, 75861,
        7110], dtype=int64)

# COSINE similarity

In [122]:
def cossim(user):
    col = mat1[user].toarray()[0]
    
    sim = col@mat1.T
    a = np.array(np.sum(mat1.power(2),axis=1)).flatten()
    b = np.sum(col**2)
    
    sim = sim/((a**0.5)*(b**0.5))
    ind = (-sim).argsort()
    
    return ind

In [123]:
cossim(75)[:10]

C:\Users\JAYANK~1\AppData\Local\Temp/ipykernel_13492/263940307.py:8: RuntimeWarning: invalid value encountered in true_divide
  sim = sim/((a**0.5)*(b**0.5))


array([    75,  90360, 103205,  60336,  82069,  95523,    176,  66532,
       131699,  29536], dtype=int64)

In [124]:
def recommend(user,number_of_movies=5):
    u1 = jacsim(user)[1]
    u2 = cossim(user)[1]
    
    col = mat1[user].toarray()[0]
    a = mat1[u1].toarray()[0]
    print("jaccard similarity",u1)
    a = a*(col==0)
    ind = (-a).argsort()
    print(movies.iloc[ind[:5]])
    
    b = mat1[u2].toarray()[0]
    print("cosine similarity",u2)
    b = b*(col==0)
    ind = (-b).argsort()
    print(movies.iloc[ind[:5]])

### Recommendations from system using similar user

In [125]:
recommend(69)

jaccard similarity 95719
      Movie_ID  Year                Name
1294      1295  1983        Strange Brew
878        879  1976     Mikey and Nicky
719        720  1989          Roger & Me
38          39  2000     Love Reinvented
721        722  2003  The Wire: Season 1
cosine similarity 115399
      Movie_ID  Year                              Name
1151      1152  1999                          Charisma
223        224  1999  Midsomer Murders: Blood Will Out
436        437  2005           Beyblade: Fierce Battle
920        921  1952             Don't Bother to Knock
1314      1315  2001                    Guns and Talks


C:\Users\JAYANK~1\AppData\Local\Temp/ipykernel_13492/263940307.py:8: RuntimeWarning: invalid value encountered in true_divide
  sim = sim/((a**0.5)*(b**0.5))


### The movies that the person has seen

In [126]:
a = mat1[69].toarray()[0]
ind = (-a).argsort()
movies.iloc[ind[:20]]

,Movie_ID,Year,Name
1291,1292,2002,Ararat
1107,1108,2000,Dr. Andrew Weil: 8 Weeks to Optimum Health & S...
479,480,1997,Flypaper
92,93,2004,They Came Back
1304,1305,2003,Thirteen
190,191,2003,X2: X-Men United
275,276,1998,Cold Harvest
87,88,1998,Record of Lodoss War: Chronicles of the Heroic...
814,815,1959,Tiger Bay
284,285,1997,The Devil's Own
